# Generative Adverserial Networks (GAN)

A GAN is a machine learning model where two neural networks compete against each other to become more accurate. This is done by having a generator attempt to create a 'counterfeit' version of real images, and a discriminator attempt to distinguish between a dataset of real images and the new fake images.

After multiple rounds of this 'competition' occur, eventually the generator gains the ability to create somewhat accurate immitations of a given data set. In this notebook, we are going to explain the basics of how a GAN works, and how to generate new samples from an already-trained generator.


<img src="GANdiagram.jpeg" style="height:300px;"/>

In [33]:
%matplotlib widget
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import sys  
sys.path.insert(0, '../my_imageprocessing/dip_utils') #This may or may not be where your dip_utils folder is

import matrix_utils
from torchvision.utils import make_grid
import numpy as np
from torchvision.transforms import ToTensor

In [24]:
def denorm(img_tensors):
    return img_tensors*stats[1][0] + stats[0][0]

def show_images(images, nmax = 64):
    fig, ax = plt.subplots(figsize =(8,8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]),nrow=8).permute(1,2,0))
            
def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break          

# The Generator

The generator is initially given latent tensors that are filled with completely random noise using `torch.randn`, this is so that there is more variation in the resulting fake images and how they are manipulated when we begin training. 

Below is the code for the generator, compared to the neural network used in `mnist_conv.ipynb` these networks are very similar, but require many more convolutional layers, as we are looking to generate images with a high amount of detail.

In [25]:
#NN Parameters
latent_size = 128
batch_size = 128
stats = (0.5,0.5,0.5),(0.5,0.5,0.5)

In [26]:
generator = nn.Sequential(
    # in: lantent_size x 1 x 1
    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride =1, padding = 0, bias = False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512x4x4
    
    nn.ConvTranspose2d(512,256, kernel_size=4, stride=2, padding = 1, bias =False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256x8x8

    nn.ConvTranspose2d(256,128, kernel_size=4,stride =2,padding = 1, bias = False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128x16x16

    nn.ConvTranspose2d(128,64, kernel_size=4, stride=2,padding=1,bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64x32x32

    nn.ConvTranspose2d(64,3,kernel_size=4,stride=2,padding=1,bias=False),
    nn.Tanh()
    # out: 3x64x64
)

## Creating Random Tensors
Below we create our batch of tensors, and run them through the generator. 

Since we have not trained it with anything, it makes sense that nothing happens...


In [27]:
random_tensors = torch.randn(batch_size, latent_size,1,1) # random latent tensors
fake_images = generator(random_tensors)

show_images(fake_images, 128)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Discriminator

The Discriminator is in charge of identifying if images are real or fake. This neural network is important in the training of the generator, but does not have much use after the training is finished.

Look in `GAN_training.ipynb` for a working example.

Below is the code we used for our discriminator, it is not used elsewhere in this notebook, but we added it for convenience :)

In [28]:
discriminator = nn.Sequential(
    # in: 3x64x64
    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias = False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),

    # out: 64x32x32
    nn.Conv2d(64,128, kernel_size=4, stride=2, padding=1,bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2,inplace = True),

    # out: 256x8x8
    nn.Conv2d(128,256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),

    nn.Conv2d(256,512, kernel_size=4,stride=2,padding=1,bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),

    # out: 512x4x4

    nn.Conv2d(512,1,kernel_size=4,stride=1, padding =0, bias = False),
    # out: 1x1x1
    nn.Flatten(),
    nn.Sigmoid()
)

# Generating images with an existing GAN
Once a GAN has been trained, the state of the generator can be saved. This state includes all of the parameters and necesary information for the generator to be used again. These states are typically saved as either `.pt` or `.pth` files and store all the necessary information to reload the training state of a neural net elsewhere.

Below we will load an already trained generator and input the random tensors we created above to generate some fake images!

## Different Generators for Different Objects
To demonstrate the results of training a GAN, we have created two generators with different data sets.

- `spaceGen.pth`: using a collection of around 7,000 image of outer space
- `carGen.pth`: using a dataset from Stanford containing 60,000+ images of cars

Try out both of them! Both generators use identical parameters except for space using 50 epochs, while cars used 25. Do the generators create realistic looking images?

In [29]:
#load our saved state into the NN
generator.load_state_dict(torch.load("carGen.pth",map_location=torch.device('cpu'))) #We do not need to power of the GPU to do this, so this forces torch to use the CPU for loading
#generator.load_state_dict(torch.load("spaceGen.pth",map_location=torch.device('cpu')))
generator.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'carGen.pth'

In [30]:
#creates more random noise and run it through the generator 
xb = torch.randn(batch_size, latent_size,1,1)
generated_images = generator(xb) #line running latent space through the generator

## Lets output the results!

In [31]:
#show the generated images
print(generated_images.shape)
show_images(generated_images, 128)

torch.Size([128, 3, 64, 64])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
#Just showing an individual image here..
show_images(generated_images[8,None,...], 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Test Questions

**1. In a GAN, what is the job of the generator and how does it interact with the discriminator?**

The generator takes in random "noise" and uses its learned model to try and create an image that will fool the discriminator. As it trains the generator improoves it's ability to fool the discriminator and eventually starts to produce realistic looking images.

**2. In a GAN, what is the job of the discriminator and how does it interact with the generator?**

The discriminator's job is to decide whener or not a given image is real or a faked image created by the generator. It learns alongside the generator to improve its acuracy which pushes the generator to become better and better at its job.

## Sources and References

**Information on GANs and how they work**

[YouTube lesson/tutorial](https://www.youtube.com/watch?v=LZov6445YAY)

**Online GAN Playgrounds/Examples**

[Polo CLub in browser GAN](https://poloclub.github.io/ganlab/)

[GAN Playground](https://reiinakano.com/gan-playground/)

**Referenced GitHub**

[github we used to build this](https://github.com/thunguyenuehk39/computer-vision-from-scratch/blob/main/Training%20Generative%20Adversarial%20Networks%20(GANs)%20in%20PyTorch.ipynb)
